In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import sys
sys.path.append('d:/flagellar/code/core')
sys.path.append('/flagellar/code/core/')
sys.path.append('/kaggle/input/my-flagellar-library/')
import flg_support as fls
import importlib
import numpy as np
import matplotlib.pyplot as plt
import time
import copy
import flg_yolo
import pandas as pd

In [5]:
p = pd.read_csv(fls.code_dir+'/../analysis/res.csv')
#p = p[p['score']>0.3]
p['intercept']=1
p

,seed,hsv_h,hsv_s,hsv_v,translate,scale,fliplr,mosaic,mixup,erasing,fix_norm_bug,score_k,score_c,score_e,intercept
0,0,1,1,1,1,1,1,1,0,0,0,0.449,NaN,NaN,1
1,0,1,1,1,1,1,0,0,1,1,0,0.587,NaN,NaN,1
2,0,1,1,1,0,0,1,1,1,1,0,0.532,NaN,NaN,1
3,0,1,1,1,1,1,1,1,1,1,0,0.525,NaN,NaN,1
4,0,0,0,0,1,1,1,1,1,1,0,0.623,NaN,NaN,1
5,0,0,0,0,0,0,0,0,0,0,0,0.084,NaN,NaN,1
6,0,1,0,0,0,1,1,1,1,1,0,0.541,0.872,0.385,1
7,1,1,1,0,1,0,0,1,0,1,0,0.455,0.846,0.264,1
8,2,0,0,1,0,1,1,0,0,0,0,0.403,0.885,0.434,1
9,3,0,0,1,1,0,0,0,0,1,0,0.498,0.866,NaN,1


In [6]:
import statsmodels.api as sm
import ISLP
import ISLP.models
X = p.drop(['seed', 'score_k', 'score_c', 'score_e'],axis=1)
y = p['score_k']
to_keep = np.logical_not(np.isnan(y))
X = X[to_keep]; y=y[to_keep];
model = sm.OLS(y,X)

In [12]:
results = model.fit_regularized(method='elastic_net', L1_wt=1, alpha=0.001)
results.params

hsv_h           0.000000
hsv_s           0.000000
hsv_v           0.041343
translate       0.075908
scale           0.119726
fliplr          0.042273
mosaic          0.049763
mixup           0.000000
erasing         0.115093
fix_norm_bug    0.000000
intercept       0.201475
dtype: float64

In [8]:
results = model.fit()
results.summary()

C:\Users\jeroe\anaconda3\envs\jupyterlab-debugger2\lib\site-packages\statsmodels\regression\linear_model.py:1966: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt(eigvals[0]/eigvals[-1])


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                score_k   R-squared:                       0.810
Model:                            OLS   Adj. R-squared:                  0.678
Method:                 Least Squares   F-statistic:                     6.150
Date:                Sat, 05 Apr 2025   Prob (F-statistic):            0.00184
Time:                        07:55:32   Log-Likelihood:                 33.882
No. Observations:                  23   AIC:                            -47.76
Df Residuals:                      13   BIC:                            -36.41
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
hsv_h           -0.0037      0.052     -0.072      0.944      -0.115       0.108
hsv_s           -0.0306      0.040     -0.769      0.455      -0.117       0.055
hsv_v            0.0365      0.040      0.923      0.373      -0.049       0.122
translate        0.0726      0.035      2.061      0.060      -0.003       0.149
scale            0.0872      0.041      2.107      0.055      -0.002       0.177
fliplr           0.0161      0.037      0.429      0.675      -0.065       0.097
mosaic           0.0446      0.045      0.993      0.339      -0.052       0.142
mixup            0.0930      0.040      2.340      0.036       0.007       0.179
erasing          0.1128      0.037      3.055      0.009       0.033       0.193
fix_norm_bug          0          0        nan        nan           0           0
intercept        0.2229      0.046      4.841      0.000       0.123       0.322
==============================================================================
Omnibus:                        6.546   Durbin-Watson:                   1.505
Prob(Omnibus):                  0.038   Jarque-Bera (JB):                4.709
Skew:                          -1.080   Prob(JB):                       0.0950
Kurtosis:                       3.497   Cond. No.                          inf
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is      0. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""